<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/Exercise%20task%201%3A%20lexical%20ambiguity(English-EWT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Execute shell command to download the file using wget
!wget https://raw.githubusercontent.com/simulate111/Introduction-to-Human-Language-Technology/main/en_ewt-ud-train.conllu


--2024-03-15 22:58:24--  https://raw.githubusercontent.com/simulate111/Introduction-to-Human-Language-Technology/main/en_ewt-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13846707 (13M) [text/plain]
Saving to: ‘en_ewt-ud-train.conllu’

en_ewt-ud-train.con 100%[===================>]  13.21M  36.8MB/s    in 0.4s    

2024-03-15 22:58:25 (36.8 MB/s) - ‘en_ewt-ud-train.conllu’ saved [13846707/13846707]



In [49]:
import nltk
from nltk.tokenize import word_tokenize

# Load the English-EWT training dataset
with open("en_ewt-ud-train.conllu", "r", encoding="utf-8") as file:
    data = file.read()

# Function to extract text from CoNLL-U format
def extract_text(data):
    lines = data.split("\n")
    tokens = []
    for line in lines:
        if line and not line.startswith("#"):  # Skip comments and empty lines
            parts = line.split("\t")
            if len(parts) >= 2:  # Ensure it's a valid token line
                tokens.append(parts[1])  # Extract the token
    return " ".join(tokens)

# Extract text from the dataset
text = extract_text(data)

# Tokenize the text using NLTK
tokens = word_tokenize(text)

# Print the tokens
print(tokens[:20])  # Print first 20 tokens as a sample


['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the']


In [50]:
import re

# Assuming 'tokens' contains the list of tokenized words

# Remove punctuations
tokenized = [re.sub(r'[^\w\s]', '', token) for token in tokens]

# Handle clitics like "n't"
tokenized = [re.sub(r"(n't)", r" \1", token) for token in tokenized]

# Remove numeric numbers
tokenized = [token for token in tokenized if not re.match(r'\d+', token)]

# Remove empty lines
tokenized = [line for line in tokenized if line.strip()]

# Remove one-letter words
tokenized = [token for token in tokenized if len(token) > 1]

# Print the updated tokenized list
print(tokenized)

# Print 10 lines
num_lines = 10  # Number of lines to print
for line in tokenized[:num_lines]:
    print(line)


['Al', 'Zaman', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', 'Ani', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', 'near', 'the', 'Syrian', 'border', 'This', 'killing', 'of', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', 'DPA', 'Iraqi', 'authorities', 'announced', 'that', 'they', 'had', 'busted', 'up', 'terrorist', 'cells', 'operating', 'in', 'Baghdad', 'Two', 'of', 'them', 'were', 'being', 'run', 'by', 'officials', 'of', 'the', 'Ministry', 'of', 'the', 'Interior', 'The', 'MoI', 'in', 'Iraq', 'is', 'equivalent', 'to', 'the', 'US', 'FBI', 'so', 'this', 'would', 'be', 'like', 'having', 'Edgar', 'Hoover', 'unwittingly', 'employ', 'at', 'high', 'level', 'members', 'of', 'the', 'Weathermen', 'bombers', 'back', 'in', 'the', 'The', 'third', 'was', 'being', 'run', 'by', 'the', 'head', 'of', 'an', 'investment', 'firm', 'You', 'wonder', 'if', 'he', 'was', 'manipulating', 'the', 'market', 'with', 'his', 

In [52]:
# Sort word counts by frequency
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Number of common words to display
num_common_words = 10

# Print the most common words
for word, count in sorted_word_counts[:num_common_words]:
    print(f"{word}: {count}")


the: 8151
to: 5077
and: 4855
of: 3589
in: 2911
is: 2178
you: 2029
that: 1981
for: 1789
it: 1588


In [55]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Increase max_length limit
nlp.max_length = 1500000  # or any value suitable for your text length

# Tokenize the text using spaCy
doc = nlp(text)

# Initialize counts for nouns and verbs
noun_count = 0
verb_count = 0

# Count nouns and verbs
for token in doc:
    if token.pos_ == 'NOUN':  # Noun
        noun_count += 1
    elif token.pos_ == 'VERB':  # Verb
        verb_count += 1

# Print counts
print("Noun count:", noun_count)
print("Verb count:", verb_count)


Noun count: 33526
Verb count: 23237


In [61]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Split the text into smaller chunks (e.g., by sentences)
chunks = text.split('.')  # You can split by other criteria as well

# Initialize dictionaries to store frequencies
noun_freq = {}
verb_freq = {}

# Process each chunk separately
for chunk in chunks:
    # Tokenize the chunk using spaCy
    doc = nlp(chunk)

    # Count nouns and verbs
    for token in doc:
        if token.pos_ == 'NOUN':  # Noun
            noun_freq[token.text] = noun_freq.get(token.text, 0) + 1
        elif token.pos_ == 'VERB':  # Verb
            verb_freq[token.text] = verb_freq.get(token.text, 0) + 1


In [60]:
# Sort noun frequencies by frequency
sorted_noun_freq = sorted(noun_freq.items(), key=lambda x: x[1], reverse=True)

# Print top 10 noun frequencies
print("Top 10 NOUN frequencies:")
for noun, freq in sorted_noun_freq[:10]:
    print(f"{noun}: {freq}")

# Sort verb frequencies by frequency
sorted_verb_freq = sorted(verb_freq.items(), key=lambda x: x[1], reverse=True)

# Print top 10 verb frequencies
print("\nTop 10 VERB frequencies:")
for verb, freq in sorted_verb_freq[:10]:
    print(f"{verb}: {freq}")


Top 10 NOUN frequencies:
time: 387
=: 312
people: 234
way: 193
place: 181
year: 174
day: 168
food: 158
service: 157
Thanks: 154

Top 10 VERB frequencies:
have: 736
get: 355
know: 338
had: 318
do: 273
go: 260
said: 233
want: 207
going: 191
see: 186


In [64]:
# Extract keys from noun_freq and verb_freq dictionaries
noun_words = set(noun_freq.keys())
verb_words = set(verb_freq.keys())

# Find common words between noun_words and verb_words
common_words = noun_words.intersection(verb_words)

# Print the number of common words
print("Number of common words between nouns and verbs:", len(common_words))


Number of common words between nouns and verbs: 791
